In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/tf_hub_obj_detection/deploy_tfhub_object_detection_on_vertex_endpoints.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/tf_hub_obj_detection/deploy_tfhub_object_detection_on_vertex_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/tf_hub_obj_detection/deploy_tfhub_object_detection_on_vertex_endpoints.ipynb">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="Vertex AI Workbench notebook"> 在 Vertex AI Workbench 中打开
    </a>
  </td> 
</table>

使用Vertex AI端点部署TensorFlow Hub目标检测模型

概述
本教程演示了如何获取一个 TensorFlow Hub 目标检测模型，添加一个预处理层，并将其部署到 Vertex AI 端点进行在线预测。

由于目标检测模型接受张量作为输入，我们将添加一个预处理层来接受jpeg字符串并解码它们。这样客户端就可以更容易地调用端点，而无需实现他们自己的 TensorFlow 逻辑。

## 模型
本教程使用的模型是从[TensorFlow Hub开源模型仓库](https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1)中获取的`CenterNet HourGlass104 Keypoints 512x512`模型。

## 目标

执行的步骤包括：
- 从 TensorFlow Hub 下载一个目标检测模型。
- 使用 @tf.function 创建一个预处理层。
- 将模型上传到 Vertex AI 的 `Models`。
- 创建一个 Vertex AI 的 `Endpoint`。
- 使用 `Python Vertex AI SDK` 和通过 `CURL` 命令行调用端点。
- 卸载端点并删除模型。

费用
本教程使用Google Cloud的计费组件：
- Vertex AI
- 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

## 安装
安装最新版本的Python用的Vertex SDK。

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip install {USER_FLAG} --upgrade google-cloud-aiplatform

安装TensorFlow。

In [ ]:
!pip install -U "tensorflow>=2.7"

重新启动内核

安装完成后，您需要重新启动笔记本内核，以便它可以找到包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

设置您的Google Cloud项目

**无论您使用哪种笔记本环境，都需要完成以下步骤。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得$300的免费信用额度来支付计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以尝试使用`gcloud`获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目 ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = ""  # @param {type:"string"}

时间戳

如果您在直播教程会话中，可能会使用共享测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用谷歌云笔记本**，您的环境已经通过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行帐户oAuth验证。

**否则**，请按照以下步骤操作：

1. 在云控制台中，转到[**创建服务帐户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐户**。

3. 在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务帐户对项目的访问权限**部分，点击**角色**下拉列表。在筛选框中输入"Vertex AI"，并选择**Vertex AI管理员**。在筛选框中输入"Storage Object Admin"，并选择**存储对象管理员**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中输入您的服务帐户密钥的路径，并将其作为`GOOGLE_APPLICATION_CREDENTIALS`变量运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

首先将模型文件上传到一个云存储桶中。使用这个模型 artifact，然后您就可以
创建 Vertex AI 模型和端点资源，以便提供
在线预测。

在下方设置您的云存储桶的名称。它必须在所有
云存储桶中是唯一的。

您还可以更改 `REGION` 变量，这将影响
笔记本中其他操作。请确保 [选择 Vertex AI 服务可用的区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。您可能
不能将多区域存储桶用于与 Vertex AI 的训练。

In [ ]:
BUCKET_NAME = ""  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

In [ ]:
print(BUCKET_NAME)

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -p $PROJECT_ID -l $REGION $BUCKET_NAME

最后，通过检查存储桶的内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

## 下载并解压模型
TensorFlow Hub中有各种物体检测模型。我们将使用`CenterNet HourGlass104 Keypoints 512x512`。

In [ ]:
# Download and extract model
!wget https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1?tf-hub-format=compressed
!tar xvzf 1?tf-hub-format=compressed
!mkdir obj_detect_model
!mv ./saved_model.pb obj_detect_model/
!mv ./variables obj_detect_model/

可视化工具
为了展示具有正确检测框、关键点和分割的图像，我们将使用TensorFlow目标检测API。为了安装它，我们将克隆存储库。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
from six import BytesIO

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

安装目标检测API

In [ ]:
%%bash

sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
pip install .

现在我们可以导入以后会用到的依赖项。

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

加载标签映射数据（用于绘图）。
标签映射将索引号对应到类别名称，因此当我们的卷积网络预测为5时，我们知道这对应飞机。在这里我们使用内部实用函数，但任何返回将整数映射到适当字符串标签的字典的函数都可以。

为简单起见，我们将从我们加载Object Detection API代码的存储库中加载。

In [ ]:
PATH_TO_LABELS = "./models/research/object_detection/data/mscoco_label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS, use_display_name=True
)
print(category_index[5])

加载模型
在这里，我们将把下载好的模型加载到内存中。

In [ ]:
model = tf.saved_model.load("obj_detect_model/")

加载一张图片并使用模型进行推理。

In [ ]:
image_path = "models/research/object_detection/test_images/image2.jpg"


def load_image_into_numpy_array(path):
    image_data = tf.io.gfile.GFile(path, "rb").read()
    image = Image.open(BytesIO(image_data))

    (width, height) = image.size
    return np.array(image.getdata()).reshape((1, height, width, 3)).astype(np.uint8)


image_np = load_image_into_numpy_array(image_path)
plt.figure(figsize=(24, 32))
plt.imshow(image_np[0])
plt.show()


results = model(image_np)
result = {key: value.numpy() for key, value in results.items()}

可视化结果

In [ ]:
COCO17_HUMAN_POSE_KEYPOINTS = [
    (0, 1),
    (0, 2),
    (1, 3),
    (2, 4),
    (0, 5),
    (0, 6),
    (5, 7),
    (7, 9),
    (6, 8),
    (8, 10),
    (5, 6),
    (5, 11),
    (6, 12),
    (11, 12),
    (11, 13),
    (13, 15),
    (12, 14),
    (14, 16),
]

label_id_offset = 0
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if "detection_keypoints" in result:
    keypoints = result["detection_keypoints"][0]
    keypoint_scores = result["detection_keypoint_scores"][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections[0],
    result["detection_boxes"][0],
    (result["detection_classes"][0] + label_id_offset).astype(int),
    result["detection_scores"][0],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.30,
    agnostic_mode=False,
    keypoints=keypoints,
    keypoint_scores=keypoint_scores,
    keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS,
)

plt.figure(figsize=(24, 32))
plt.imshow(image_np_with_detections[0])
plt.show()

## 为Vertex AI服务创建一个预处理函数。
模型期望以numpy数组作为输入。这为我们的端点创建了两个问题：
* Vertex AI公共端点的最大请求大小为1.5 MB。图片远大于此大小。
* 对于使用其他编程语言的客户端来构建请求会更加困难。

通过构建一个预处理函数并将其附加到我们的模型，这两个限制可以得到解决。

我们将创建一个预处理函数，该函数接收一个jpeg编码的图片，将其调整大小为模型所需的最小输入，并将这个预处理输入传递给模型。然后我们将保存带有预处理函数的模型，该模型将准备好上传到我们的Vertex AI端点。

图片将作为一个base64编码的jpeg字符串传递给我们的端点。

In [ ]:
VERTEX_MODEL_PATH = "obj_detect_model_vertex/"


def _preprocess(bytes_inputs):
    decoded = tf.io.decode_jpeg(bytes_inputs, channels=3)
    resized = tf.image.resize(decoded, size=(512, 512))
    return tf.cast(resized, dtype=tf.uint8)


def _get_serve_image_fn(model):
    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def serve_image_fn(bytes_inputs):
        decoded_images = tf.map_fn(_preprocess, bytes_inputs, dtype=tf.uint8)
        return model(decoded_images)

    return serve_image_fn


signatures = {
    "serving_default": _get_serve_image_fn(model).get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string)
    )
}

tf.saved_model.save(model, VERTEX_MODEL_PATH, signatures=signatures)

我们将使用`saved_model_cli`命令在原模型和Vertex AI准备的模型上验证输入是否正确修改。

`serving_default`签名的结果应该如下。

原模型：

```
signature_def['serving_default']:
  给定的SavedModel SignatureDef 包含以下输入：
    inputs['input_tensor'] tensor_info:
        dtype: DT_UINT8
        shape: (1, -1, -1, 3)
        name: serving_default_input_tensor:0
```

Vertex AI模型：

```
signature_def['serving_default']:
  给定的SavedModel SignatureDef 包含以下输入：
    inputs['bytes_inputs'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_bytes_inputs:0
```

In [ ]:
!saved_model_cli show --dir obj_detect_model --all

In [ ]:
!saved_model_cli show --dir obj_detect_model_vertex --all

让我们通过传递一个 base 64 编码的 jpeg 图像来测试预处理函数。

In [ ]:
vertex_model = tf.saved_model.load(VERTEX_MODEL_PATH)

In [ ]:
import base64


def encode_image(image):
    with open(image, "rb") as image_file:
        encoded_string = base64.urlsafe_b64encode(image_file.read()).decode("utf-8")
    return encoded_string


results = vertex_model([_preprocess(tf.io.decode_base64(encode_image(image_path)))])

查看结果

In [ ]:
# different object detection models have additional results
# all of them are explained in the documentation
result = {key: value.numpy() for key, value in results.items()}

label_id_offset = 0
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if "detection_keypoints" in result:
    keypoints = result["detection_keypoints"][0]
    keypoint_scores = result["detection_keypoint_scores"][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections[0],
    result["detection_boxes"][0],
    (result["detection_classes"][0] + label_id_offset).astype(int),
    result["detection_scores"][0],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.30,
    agnostic_mode=False,
    keypoints=keypoints,
    keypoint_scores=keypoint_scores,
    keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS,
)

plt.figure(figsize=(24, 32))
plt.imshow(image_np_with_detections[0])
plt.show()

创建一个顶点AI端点
在这个部分，我们将把模型上传到Google Cloud Storage，并在Vertex AI中引用它用于端点部署。

In [ ]:
!gsutil cp -r $VERTEX_MODEL_PATH $BUCKET_NAME/obj_detection_model_vertex

In [ ]:
!gsutil ls $BUCKET_NAME

在Vertex AI中创建一个模型

In [ ]:
!gcloud ai models upload \
--region=us-central1 \
--project=$PROJECT_ID \
--display-name=object-detection \
--container-image-uri=us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest \
--artifact-uri=$BUCKET_NAME/obj_detection_model_vertex

创建终端点

In [ ]:
!gcloud ai endpoints create \
--project=$PROJECT_ID \
--region=$REGION \
--display-name=object-detection-endpoint

检索MODEL_ID和ENDPOINT_ID

In [ ]:
%%bash -s "$REGION" "$PROJECT_ID" --out MODEL_ID
MODEL_ID=`gcloud ai models list --region=$1 --project=$2 | grep object-detection`
echo $MODEL_ID | cut -d' ' -f1 | tr -d '\n'

In [ ]:
%%bash -s "$REGION" "$PROJECT_ID" --out ENDPOINT_ID
ENDPOINT_ID=`gcloud ai endpoints list --region=$1 --project=$2 | sed -n 2p`
echo $ENDPOINT_ID | cut -d' ' -f1 | tr -d '\n'

In [ ]:
!gcloud ai endpoints deploy-model $ENDPOINT_ID \
--project=$PROJECT_ID \
--region=$REGION \
--model=$MODEL_ID \
--display-name=object-detection-endpoint \
--traffic-split=0=100

将请求写入一个json文件，并使用Curl调用端点。

首先，我们需要减少图像的内存占用。截至2022年2月，Vertex AI端点的最大请求大小为1.5mb。这样做是为了确保在高负载时期，端点后面的容器不会崩溃。

In [ ]:
import os

print(os.stat(image_path).st_size)

im = Image.open(image_path)
im.save("image2.jpg", quality=95)
print(os.stat("image2.jpg").st_size)

In [ ]:
!echo {"\""instances"\"" : [{"\""bytes_inputs"\"" : {"\""b64"\"" : "\""$(base64 "image2.jpg")"\""}}]} > instances.json

In [ ]:
!curl POST  \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/us-central1/endpoints/$ENDPOINT_ID:predict \
-d @instances.json > results.json

使用Vertex SDK进行预测
Vertex SDK具有方便的方法来调用端点以进行预测。
首先，我们从模型中获取用于服务的输入。这是端点期望的base64编码图像的密钥。

In [ ]:
# Get the input key
serving_input = list(
    vertex_model.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving input :", serving_input)

加载一个端点对象。

In [ ]:
from google.cloud import aiplatform

aip_endpoint_name = (
    f"projects/{PROJECT_ID}/locations/us-central1/endpoints/{ENDPOINT_ID}"
)
endpoint = aiplatform.Endpoint(aip_endpoint_name)

In [ ]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


# Endpoints will do the base64 decoding, so we change the function to encode the image a bit.
def encode_image_bytes(image_path):
    bytes = tf.io.read_file(image_path)
    return base64.b64encode(bytes.numpy()).decode("utf-8")


instances_list = [{serving_input: {"b64": encode_image_bytes("image2.jpg")}}]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]
results = endpoint.predict(instances=instances)

查看结果

In [ ]:
# different object detection models have additional results
# all of them are explained in the documentation
prediction_results = results.predictions[0]
result = {key: np.array([value]) for key, value in prediction_results.items()}

label_id_offset = 0
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if "detection_keypoints" in result:
    keypoints = result["detection_keypoints"][0]
    keypoint_scores = result["detection_keypoint_scores"][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections[0],
    result["detection_boxes"][0],
    (result["detection_classes"][0] + label_id_offset).astype(int),
    result["detection_scores"][0],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.30,
    agnostic_mode=False,
    keypoints=keypoints,
    keypoint_scores=keypoint_scores,
    keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS,
)

plt.figure(figsize=(24, 32))
plt.imshow(image_np_with_detections[0])
plt.show()

整理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
%%bash -s "$ENDPOINT_ID" "$REGION" "$PROJECT_ID" --out ENDPOINT_MODEL_ID
ENDPOINT_MODEL_ID=$(gcloud ai endpoints describe $1 --region=$2 --project=$3 | grep "id:")
ENDPOINT_MODEL_ID=`echo $ENDPOINT_MODEL_ID | cut -d' ' -f2`
echo $ENDPOINT_MODEL_ID | tr -d "'"

In [ ]:
# Undeploy endpoint
! gcloud ai endpoints undeploy-model $ENDPOINT_ID \
--project=$PROJECT_ID \
--region=$REGION \
--deployed-model-id=$ENDPOINT_MODEL_ID \

# Delete endpoint resource
! gcloud ai endpoints delete $ENDPOINT_ID \
--project=$PROJECT_ID \
--region=$REGION \
--quiet

# Delete model resource
! gcloud ai models delete $MODEL_ID \
--project=$PROJECT_ID \
--region=$REGION \
--quiet

# Delete Cloud Storage objects that were created
#! gsutil -m rm -r $BUCKET_NAME